## Predicting logP with RNNs and SMILES strings 

This notebook demonstrates how to build predictive recurrent neural network for SMILES strings. We will build regression model for logP with OpenChem Toolkit (https://github.com/Mariewelt/OpenChem)

In [ ]:
# Cloning OpenChem. Comment this line if you already cloned the repository
! git clone https://github.com/Mariewelt/OpenChem.git

## Imports

In [2]:
import sys
import os

In [3]:
sys.path.append('./OpenChem')

In [4]:
from openchem.models.Smiles2Label import Smiles2Label
from openchem.modules.encoders.rnn_encoder import RNNEncoder
from openchem.modules.mlp.openchem_mlp import OpenChemMLP
from openchem.data.smiles_data_layer import SmilesDataset
from openchem.data.utils import save_smiles_property_file
from openchem.data.utils import create_loader
from openchem.models.openchem_model import build_training, fit, evaluate

In [5]:
import torch.nn as nn
from torch.optim import RMSprop, SGD, Adam
from torch.optim.lr_scheduler import ExponentialLR, StepLR
import torch.nn.functional as F

In [6]:
from sklearn.metrics import r2_score
import pandas as pd
import copy
import pickle

## Reading data

In [7]:
from openchem.data.utils import read_smiles_property_file
data = read_smiles_property_file('./data/logP_labels.csv', 
                                 cols_to_read=[1, 2])
smiles = data[0]
labels = data[1]

In [8]:
from openchem.data.utils import get_tokens
tokens, _, _ = get_tokens(smiles)
tokens = tokens + ' '

## Model architecture

Here we define the architecture of our Recurrent Neural Network (RNN). We will use 2 LSTM layers. For more details on how to build models with OpenChem, visit: https://mariewelt.github.io/OpenChem/

In [9]:
import torch
from openchem.utils.utils import identity
from openchem.modules.embeddings.basic_embedding import Embedding
model_object = Smiles2Label

model_params = {
    'use_cuda': True,
    'random_seed': 42,
    'world_size': 1,
    'task': 'regression',
    'data_layer': SmilesDataset,
    'use_clip_grad': False,
    'batch_size': 128,
    'num_epochs': 51,
    'logdir': './logs/logp_logs',
    'print_every': 1,
    'save_every': 5,
    #'train_data_layer': train_dataset,
    #'val_data_layer': test_dataset,
    'eval_metrics': r2_score,
    'criterion': nn.MSELoss(),
    'optimizer': Adam,
    'optimizer_params': {
        'lr': 0.005,
    },
    'lr_scheduler': StepLR,
    'lr_scheduler_params': {
        'step_size': 15,
        'gamma': 0.8
    },
    'embedding': Embedding,
    'embedding_params': {
        'num_embeddings': len(tokens),
        'embedding_dim': 128,
        'padding_idx': tokens.index(' ')
    },
    'encoder': RNNEncoder,
    'encoder_params': {
        'input_size': 128,
        'layer': "LSTM",
        'encoder_dim': 128,
        'n_layers': 2,
        'dropout': 0.8,
        'is_bidirectional': False
    },
    'mlp': OpenChemMLP,
    'mlp_params': {
        'input_size': 128,
        'n_layers': 2,
        'hidden_size': [128, 1],
        'activation': [F.relu, identity],
        'dropout': 0.0
    }
}

In [10]:
try:
    os.stat(model_params['logdir'])
except:
    os.mkdir(model_params['logdir'])

## Initializing data splitter for cross validation

In [11]:
from sklearn.model_selection import KFold
cross_validation_split = KFold(n_splits=5, shuffle=True)

In [12]:
data = cross_validation_split.split(smiles, labels)

## Training cross-validated models

In [13]:
import os
i = 0
models = []
results = []
for split in data:
    print('Cross validation, fold number ' + str(i) + ' in progress...')
    train, test = split
    X_train = smiles[train]
    y_train = labels[train]
    X_test = smiles[test]
    y_test = labels[test]
    save_smiles_property_file('./data/logp_train_fold' + str(i) + '.smi', 
                              X_train, y_train.reshape(-1, 1))
    save_smiles_property_file('./data/logp_test_fold' + str(i) + '.smi', 
                              X_test, y_test.reshape(-1, 1))

    train_dataset = SmilesDataset('./data/logp_train_fold' + str(i) + '.smi',
                           delimiter=',', cols_to_read=[0, 1], tokens=tokens)
    test_dataset = SmilesDataset('./data/logp_test_fold' + str(i) + '.smi',
                       delimiter=',', cols_to_read=[0, 1], tokens=tokens)
    model_params['train_data_layer'] = train_dataset
    model_params['val_data_layer'] = test_dataset
    model_params['logdir'] = './logs/logp_logs/fold' + str(i)  
    logdir = model_params['logdir']
    ckpt_dir = logdir + '/checkpoint/'
    try:
        os.stat(ckpt_dir)
    except:
        os.mkdir(logdir)
        os.mkdir(ckpt_dir)
    train_loader = create_loader(train_dataset,
                             batch_size=model_params['batch_size'],
                             shuffle=True,
                             num_workers=4,
                             pin_memory=True,
                             sampler=None)
    val_loader = create_loader(test_dataset,
                           batch_size=model_params['batch_size'],
                           shuffle=False,
                           num_workers=1,
                           pin_memory=True)
    models.append(model_object(params=model_params).cuda())
    criterion, optimizer, lr_scheduler = build_training(models[i], model_params)
    results.append(fit(models[i], lr_scheduler, train_loader, optimizer, criterion,
        model_params, eval=True, val_loader=val_loader))
    
    i = i+1

Cross validation, fold number 0 in progress...
TRAINING: [Time: 0m 2s, Epoch: 0, Progress: 0%, Loss: 2.2958]
EVALUATION: [Time: 0m 0s, Loss: 1.3561, Metrics: 0.5885]
TRAINING: [Time: 0m 5s, Epoch: 1, Progress: 1%, Loss: 1.0152]
EVALUATION: [Time: 0m 0s, Loss: 0.5825, Metrics: 0.8231]
TRAINING: [Time: 0m 7s, Epoch: 2, Progress: 3%, Loss: 0.7186]
EVALUATION: [Time: 0m 0s, Loss: 0.7800, Metrics: 0.7640]
TRAINING: [Time: 0m 10s, Epoch: 3, Progress: 5%, Loss: 0.6039]
EVALUATION: [Time: 0m 0s, Loss: 0.5324, Metrics: 0.8384]
TRAINING: [Time: 0m 12s, Epoch: 4, Progress: 7%, Loss: 0.5563]
EVALUATION: [Time: 0m 0s, Loss: 0.4946, Metrics: 0.8499]
TRAINING: [Time: 0m 15s, Epoch: 5, Progress: 9%, Loss: 0.5006]
EVALUATION: [Time: 0m 0s, Loss: 0.5587, Metrics: 0.8310]
TRAINING: [Time: 0m 18s, Epoch: 6, Progress: 11%, Loss: 0.4807]
EVALUATION: [Time: 0m 0s, Loss: 0.4322, Metrics: 0.8676]
TRAINING: [Time: 0m 20s, Epoch: 7, Progress: 13%, Loss: 0.4437]
EVALUATION: [Time: 0m 0s, Loss: 0.7048, Metrics: 0.

TRAINING: [Time: 0m 46s, Epoch: 16, Progress: 31%, Loss: 0.2810]
EVALUATION: [Time: 0m 0s, Loss: 0.2730, Metrics: 0.9221]
TRAINING: [Time: 0m 48s, Epoch: 17, Progress: 33%, Loss: 0.2624]
EVALUATION: [Time: 0m 0s, Loss: 0.3173, Metrics: 0.9110]
TRAINING: [Time: 0m 51s, Epoch: 18, Progress: 35%, Loss: 0.2480]
EVALUATION: [Time: 0m 0s, Loss: 0.2876, Metrics: 0.9191]
TRAINING: [Time: 0m 54s, Epoch: 19, Progress: 37%, Loss: 0.2460]
EVALUATION: [Time: 0m 0s, Loss: 0.2733, Metrics: 0.9229]
TRAINING: [Time: 0m 57s, Epoch: 20, Progress: 39%, Loss: 0.2375]
EVALUATION: [Time: 0m 0s, Loss: 0.2693, Metrics: 0.9229]
TRAINING: [Time: 0m 59s, Epoch: 21, Progress: 41%, Loss: 0.2377]
EVALUATION: [Time: 0m 0s, Loss: 0.2973, Metrics: 0.9142]
TRAINING: [Time: 1m 2s, Epoch: 22, Progress: 43%, Loss: 0.2322]
EVALUATION: [Time: 0m 0s, Loss: 0.3000, Metrics: 0.9147]
TRAINING: [Time: 1m 5s, Epoch: 23, Progress: 45%, Loss: 0.2269]
EVALUATION: [Time: 0m 0s, Loss: 0.3450, Metrics: 0.9016]
TRAINING: [Time: 1m 7s, Ep

EVALUATION: [Time: 0m 0s, Loss: 0.2644, Metrics: 0.9198]
TRAINING: [Time: 1m 30s, Epoch: 33, Progress: 64%, Loss: 0.1652]
EVALUATION: [Time: 0m 0s, Loss: 0.2545, Metrics: 0.9226]
TRAINING: [Time: 1m 33s, Epoch: 34, Progress: 66%, Loss: 0.1634]
EVALUATION: [Time: 0m 0s, Loss: 0.2569, Metrics: 0.9213]
TRAINING: [Time: 1m 36s, Epoch: 35, Progress: 68%, Loss: 0.1751]
EVALUATION: [Time: 0m 0s, Loss: 0.3257, Metrics: 0.9032]
TRAINING: [Time: 1m 38s, Epoch: 36, Progress: 70%, Loss: 0.1676]
EVALUATION: [Time: 0m 0s, Loss: 0.2472, Metrics: 0.9251]
TRAINING: [Time: 1m 41s, Epoch: 37, Progress: 72%, Loss: 0.1555]
EVALUATION: [Time: 0m 0s, Loss: 0.2598, Metrics: 0.9215]
TRAINING: [Time: 1m 44s, Epoch: 38, Progress: 74%, Loss: 0.1619]
EVALUATION: [Time: 0m 0s, Loss: 0.3513, Metrics: 0.8950]
TRAINING: [Time: 1m 46s, Epoch: 39, Progress: 76%, Loss: 0.1661]
EVALUATION: [Time: 0m 0s, Loss: 0.2701, Metrics: 0.9181]
TRAINING: [Time: 1m 49s, Epoch: 40, Progress: 78%, Loss: 0.1545]
EVALUATION: [Time: 0m 0s

TRAINING: [Time: 2m 14s, Epoch: 49, Progress: 96%, Loss: 0.1328]
EVALUATION: [Time: 0m 0s, Loss: 0.2590, Metrics: 0.9201]
TRAINING: [Time: 2m 16s, Epoch: 50, Progress: 98%, Loss: 0.1372]
EVALUATION: [Time: 0m 0s, Loss: 0.2343, Metrics: 0.9275]
Cross validation, fold number 4 in progress...
TRAINING: [Time: 0m 1s, Epoch: 0, Progress: 0%, Loss: 1.9943]
EVALUATION: [Time: 0m 0s, Loss: 1.5741, Metrics: 0.5421]
TRAINING: [Time: 0m 4s, Epoch: 1, Progress: 1%, Loss: 0.9260]
EVALUATION: [Time: 0m 0s, Loss: 0.6505, Metrics: 0.8077]
TRAINING: [Time: 0m 6s, Epoch: 2, Progress: 3%, Loss: 0.7138]
EVALUATION: [Time: 0m 0s, Loss: 0.5684, Metrics: 0.8289]
TRAINING: [Time: 0m 8s, Epoch: 3, Progress: 5%, Loss: 0.5905]
EVALUATION: [Time: 0m 0s, Loss: 0.5171, Metrics: 0.8448]
TRAINING: [Time: 0m 10s, Epoch: 4, Progress: 7%, Loss: 0.5089]
EVALUATION: [Time: 0m 0s, Loss: 0.4918, Metrics: 0.8532]
TRAINING: [Time: 0m 12s, Epoch: 5, Progress: 9%, Loss: 0.4905]
EVALUATION: [Time: 0m 0s, Loss: 0.4735, Metrics: 0

## Evaluating the models

In [14]:
import numpy as np
rmse = []
r2_score = []
for i in range(5):
    test_dataset = SmilesDataset('./data/logp_test_fold' + str(i) + '.smi',
                                 delimiter=',', cols_to_read=[0, 1], tokens=tokens)
    val_loader = create_loader(test_dataset,
                               batch_size=model_params['batch_size'],
                               shuffle=False,
                               num_workers=1,
                               pin_memory=True)
    metrics = evaluate(models[i], val_loader, criterion)
    rmse.append(np.sqrt(metrics[0]))
    r2_score.append(metrics[1])

EVALUATION: [Time: 0m 0s, Loss: 0.2627, Metrics: 0.9192]
EVALUATION: [Time: 0m 0s, Loss: 0.2313, Metrics: 0.9334]
EVALUATION: [Time: 0m 0s, Loss: 0.2490, Metrics: 0.9246]
EVALUATION: [Time: 0m 0s, Loss: 0.2343, Metrics: 0.9275]
EVALUATION: [Time: 0m 0s, Loss: 0.2619, Metrics: 0.9223]


In [15]:
print("Cross-validated RMSE: ",  np.mean(rmse))
print("Cross-validated R^2 score: ", np.mean(r2_score))

Cross-validated RMSE:  0.49763956473190146
Cross-validated R^2 score:  0.9254044340191147
